# 🦜🔗 LangChain agent interacting with gmail.

Langchain stellt eine ganze Palette an Toolkits bereit die es Agenten ermöglicht, mit externen Services wie zum Beispiel Gmail, Office365, etc. zu interagieren.
Ein Toolkit ist im Grunde eine Sammlung mehrerer Tools, die man gesammelt einem Agenten zur Verfügung stellt werden können.

In diesem Beispiel wollen wir uns das GmailToolkit ansehen.


Zuerst müssen wird das Toolkit konfigurieren, sodass es per Gmail API mit unserem Google Konto kommunizieren kann.


In [ ]:
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)

Jetzt initialisieren wird das Toolkit und übergeben die vorher konfigurierten API Settings.

Im Output sehen wir, welche Tools über das Toolkit bereit gestellt werden.


In [ ]:
from langchain_community.agent_toolkits import GmailToolkit
from langgraph.prebuilt import ToolExecutor

toolkit = GmailToolkit(api_resource=api_resource)
tools = toolkit.get_tools()
tools_executor = ToolExecutor(tools)
tools

Letzlich erstellen wir einen Langchain Graphen. Hier nutzen wir wieder einen prebuilt Graphen, den wir in diesem Schritt auch ausführen.


In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

from helpers import llm

prompt = SystemMessage(content="You are an email writer assistant.")
app = create_react_agent(llm(), tools, state_modifier=prompt)

inputs = {
    "messages": [
        HumanMessage(
            content="""Create an email draft to donald@trumppp.comm about a contract building a pyramid in Egypt.
            Please argue why we are the best company for the job.
            Under no circumstances may you send the message, however. """
        )
    ]
}


app.invoke(inputs)